# Local Function Review

In [1]:
def outer_function():
    message = "Hi"
    def inner_function():
        print(message)
        
    return inner_function()

In [3]:
#Test it
outer_function()

Hi


In [5]:
def outer_function():
    message = "Hi"
    def inner_function():
        print(message)
        
    # Now return the function. No ()
    return inner_function

In [6]:
my_func= outer_function()
my_func

<function __main__.outer_function.<locals>.inner_function>

In [7]:
# Run it
my_func()

Hi


In [8]:
my_func()

Hi


In [9]:
# Now add some variables
def outer_function(msg):
    message = msg
    def inner_function():
        print(message)
        
    # Now return the function. No ()
    return inner_function

In [11]:
hi_func = outer_function("Hi")
bye_func = outer_function("Bye")

# Run it
hi_func()
bye_func()


Hi
Bye


In [12]:
# Now add some variables
# Remove intermediate members
def outer_function(msg):
    def inner_function():
        print(msg)
        
    # Now return the function. No ()
    return inner_function

In [13]:
hi_func = outer_function("Hi")
bye_func = outer_function("Bye")

# Run it
hi_func()
bye_func()

Hi
Bye


## Decorators
- A function that takes another function as argument
- Adds some functionality
- Returns another function
- All of this without altering the original function.

In [14]:
def decorator_function(original_function):
    def wrapper_function():
        return original_function()
    # Return the function (do not call the function)
    return wrapper_function  # no parenthesis 

def display():
    print("Display function ran")

In [15]:
# Test it
decorated_display = decorator_function(display)
decorated_display

<function __main__.decorator_function.<locals>.wrapper_function>

In [16]:
decorated_display()

Display function ran


In [17]:
def decorator_function(original_function):
    def wrapper_function():
        print("Wrapper executed this before {}".format(original_function.__name__))
        return original_function()
    # Return the function (do not call the function)
    return wrapper_function  # no parenthesis 

def display():
    print("Display function ran")

In [19]:
# Test it
decorated_display = decorator_function(display)
decorated_display()

Wrapper executed this before display
Display function ran


## Now decorate your functions
@Decorator_Function_Name

In [20]:
def decorator_function(original_function):
    def wrapper_function():
        print("Wrapper executed this before {}".format(original_function.__name__))
        return original_function()
    # Return the function (do not call the function)
    return wrapper_function  # no parenthesis 

@decorator_function
def display():
    print("Display function ran")

In [21]:
# Test it
display()

Wrapper executed this before display
Display function ran


This will not work if our original function takes arguments

In [24]:
@decorator_function
def display_info(name, age):
    print("display_info ran with arguments ({},{})".format(name, age))
    
display_info("Mario", 21)

TypeError: wrapper_function() takes 0 positional arguments but 2 were given

We can fix this with **\*args** and **\*\*kwargs** 

In [26]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print("Wrapper executed this before {}".format(original_function.__name__))
        return original_function(*args, **kwargs) 
    # Return the function (do not call the function)
    return wrapper_function  # no parenthesis 

@decorator_function
def display():
    print("Display function ran")
    
@decorator_function
def display_info(name, age):
    print("display_info ran with arguments ({},{})".format(name, age))

In [27]:
display()

Wrapper executed this before display
Display function ran


In [28]:
display_info("Mario", 21)

Wrapper executed this before display_info
display_info ran with arguments (Mario,21)


### Example: escape unicode

In [29]:
def escape_unicode(f):
    def wrap(*args, **kwargs):
        x = f(*args, **kwargs)
        return ascii(x)
    return wrap

@escape_unicode 
def mexico_city():
    # Alt + 130 for é Ü 
    return "México "

In [30]:
mexico_city()

"'M\\xe9xico '"

#### What can be a Decorator?
- Class Objects, which can be callable with the **dunder call**
- functions as decorators

### Classes as Decorators

In [31]:
class DecoratorClass(object):
    def __init__(self,original_function): # 
        self.original_function = original_function #
        
    def __call__(self, *args, **kwargs):
        print("Call method executed this before {}".format(self.original_function))
        return self.original_function(*args, **kwargs)
        
@DecoratorClass
def display():
    print("Display function ran")
    
@DecoratorClass
def display_info(name, age):
    print("display_info ran with arguments ({},{})".format(name, age))

In [32]:
# test it
display()
display_info("John", 32)

Call method executed this before <function display at 0x000001701D7642F0>
Display function ran
Call method executed this before <function display_info at 0x000001701D758598>
display_info ran with arguments (John,32)


### Instances as Decorators
A class instance.

In [39]:
class Trace:
    def __init__(self):
        self._enable = True
        
    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self._enable:
                print("Calling {}".format(f.__name__))
            return f(*args, **kwargs)
        return wrap 

# Create an instance
tracer = Trace()
# Instance as decorator 
@tracer
def rotate_list(l): # l 1
    return l[1:] + [l[0]] 

Unlike our previous example, the **class object itself is not the decorator**. Rather, instances of trace can be used as decorators. 

In [42]:
# test it
l1=[1, 2, 3]
print(l1)
l1 = rotate_list(l1) 
print(l1)

[1, 2, 3]
Calling rotate_list
[2, 3, 1]


In [38]:
# turn off the enable key from the instance
tracer._enable = False
l1=[1, 2, 3]
l1 = rotate_list(l1) 

Calling rotate_list


## Back to Function Decorators

In [43]:
import time

def my_timer(original_func):
    """
    Helps you keep track of the login part of a function.
    """
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_func(*args, **kwargs)
        t2 = time.time()
        print("{} ran in: {}".format(original_func.__name__, t2))
        return result
    
    return wrapper

# Test it
@my_timer
def display_info(name, age):
    time.sleep(1)
    print("Display_info ran with arguments ({}, {})".format(name, age))

In [45]:
# test it
display_info("mario", 21)

Display_info ran with arguments (mario, 21)
display_info ran in: 1495121881.1740568


## Multiple decorators

In [49]:
def my_logger(original_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(original_func.__name__), level=logging.INFO)
    def wrapper(*args, **kwargs):
        logging.info("Ran with args: {}, and kwargs: {}".format(args, kwargs))
        return original_func(*args, **kwargs)
    return wrapper 

@my_logger
def display_info(name, age):
    print("Display_info ran with arguments ({}, {})".format(name, age))

In [50]:
# Test it
display_info("Weber", 125)

Display_info ran with arguments (Weber, 125)


In [51]:
# multiple decorators
@my_timer
@my_logger
def display_info(name, age):
    time.sleep(1)
    print("Display_info ran with arguments ({}, {})".format(name, age))
    
# Test it
display_info("mario", 21)

Display_info ran with arguments (mario, 21)
wrapper ran in: 1495122729.5359278


In [52]:
# multiple decorators
@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print("Display_info ran with arguments ({}, {})".format(name, age))
    
# Test it
display_info("mario", 21)

Display_info ran with arguments (mario, 21)
display_info ran in: 1495122812.4028914


Wrap everything

In [55]:
import time
from functools import wraps

def my_logger(original_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(original_func.__name__), level=logging.INFO)
    
    @wraps(original_func)
    def wrapper(*args, **kwargs):
        logging.info("Ran with args: {}, and kwargs: {}".format(args, kwargs))
        return original_func(*args, **kwargs)
    return wrapper 

def my_timer(original_func):
    @wraps(original_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_func(*args, **kwargs)
        t2 = time.time()
        print("{} ran in: {}".format(original_func.__name__, t2))
        return result   
    return wrapper

# multiple decorators
@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print("Display_info ran with arguments ({}, {})".format(name, age))
    
# Test it
display_info("mario", 21)    

Display_info ran with arguments (mario, 21)
display_info ran in: 1495123243.5474715
